## 0. Introduction

The purpose of this notebook is to explore exotic options with reference to chapter 7 from *Stochastic Calculus for Finance II Continuous-Time Models* (Shreve, 2008) and [this GitHub page](https://github.com/cantaro86/Financial-Models-Numerical-Methods/blob/master/2.2%20Exotic%20options.ipynb) (Cantarutti, 2019).

## 1. Barrier Options

The European calls and puts considered thus far are called *vanilla* or even *plain vanilla* options. Their payoffs depend only on the final value of the underlying asset. Options whose payoffs depend on the path of the underlying asset are called *path-dependent* or *exotic* options.

Barrier options are one such example and there are several different types. Some "knock-out" when the underlying asset price crosses a barrier (i.e., they become worthless). If the underlying asset price begins below the barrier and must cross above it to cause the knock-out, the option is said to be *up-and-out*. A *down-and-out* option has the barrier below the initial asset price and knocks out if the asset price falls below the barrier.

Other options "knock-in" at a barrier (i.e., they pay off zero unless they cross a barrier). Knock-in options also fall into two classes, *up-and-in* and *down-and-in*. The payoff at expiration for barrier options is typically either that of a put or a call. 



### 1.1 Up-and-Out Call

Our underlying risky asset is geometric Brownian motion 

$$ dS(t) = r S(t) \, dt + \sigma S(t) \, d\widetilde{W}(t), $$

where $\widetilde{W}(t), 0 \leq t \leq T$, is a Brownian motion under the risk-neutral measure $\widetilde{\mathbb{P}}$. Consider a European call, expiring at time $T$, with strike price $K$ and up-and-out barrier $B$. We assume $K < B$; otherwise, the option must knock out in order to be in the money and hence could only pay off zero. The solution to the SDE for the asset price is

$$ S(t) = S(0) e^{\sigma \widetilde{W}(t) + (r - \frac{1}{2} \sigma^2)t} = S(0) e^{\sigma \widehat{W}(t)}, $$

where $\widehat{W}(t) = \alpha t + \widetilde{W}(t)$, and

$$ a = \frac{1}{\sigma} \left(r - \frac{1}{2} \sigma^2 \right). $$

We define $\widehat{M}(T) = \max_{0 \leq t \leq T} \widehat{W}(t)$, so

$$ \max_{0 \leq t \leq T} S(t) = S(0) e^{\sigma \widehat{M}(T)}. $$

The option knocks out if and only if $S(0)e^{\sigma \widehat{M}(T)} > B$; if $S(0)e^{\sigma \widehat{M}(T)} \leq B$, the option pays off

$$ (S(T) - K)^+ = (S(0)e^{\sigma \widehat{W}(T)} - K)^+. $$

In other words, the payoff of the option is

$$
\begin{aligned}
V(T) &= (S(0)e^{\sigma \widehat{W}(T)} - K)^+ \mathbb{I}_{\{S(0)e^{\sigma \widehat{M}(T)} \leq B \}} \\
     &= (S(0)e^{\sigma \widehat{W}(T)} - K) \mathbb{I}_{\{S(0)e^{\sigma \widehat{W}(T)} \geq K, S(0)e^{\sigma \widehat{M}(T)} \leq B \}} \\
     &= (S(0)e^{\sigma \widehat{W}(T)} - K) \mathbb{I}_{\{\widehat{W}(T) \geq k, \widehat{M}(T) \leq b \}},
\end{aligned}
$$

where

$$ k = \frac{1}{\sigma} \log \frac{K}{S(0)}, \quad b = \frac{1}{\sigma} \log \frac{B}{S(0)}. $$

Let's consider an example in Python.

In [1]:
import numpy as np
import scipy.stats as ss

In [2]:
# Define parameters
S0 = 100.0     # initial asset price
K = 100.0      # strike price
B = 115.0      # barrier price
T = 1.0        # time to maturity
r = 0.04       # risk-free rate per unit T
sigma = 0.2    # volatility per unit T

In [3]:
# Closed formula
d1 = lambda t, x: 1 / (sigma * np.sqrt(t)) * (np.log(x) + (r + sigma**2 / 2) * t)
d2 = lambda t, x: 1 / (sigma * np.sqrt(t)) * (np.log(x) + (r - sigma**2 / 2) * t)

closed_barrier_u = (
    S0 * (ss.norm.cdf(d1(T, S0 / K)) - ss.norm.cdf(d1(T, S0 / B)))
    - np.exp(-r * T) * K * (ss.norm.cdf(d2(T, S0 / K)) - ss.norm.cdf(d2(T, S0 / B)))
    - B * (S0 / B) ** (-2 * r / sigma**2) * (ss.norm.cdf(d1(T, B**2 / (S0 * K))) - ss.norm.cdf(d1(T, B / S0)))
    + np.exp(-r * T) * K * (S0 / B) ** (-2 * r / sigma**2 + 1) * (ss.norm.cdf(d2(T, B**2 / (S0 * K))) - ss.norm.cdf(d2(T, B / S0)))
)

print(f"The price of the Up-and-Out call option by closed formula is: {closed_barrier_u:.3f}")

The price of the Up-and-Out call option by closed formula is: 0.481


In [4]:
# Monte Carlo method
N = 252
dt = T / N
paths = 100000
S = np.zeros((N+1, paths))
S[0, :] = S0

beta1 = 0.5826                                      # correction factor to price discrete options using continuous formulas
B_adj = B * np.exp(-beta1 * np.sqrt(dt) * sigma)    # apply barrier correction

Z = ss.norm.rvs(loc=0, scale=1, size=(N,paths), random_state=42)
for i in range(1, N+1):
    S[i, :] = S[i-1, :] * np.exp((r - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * Z[i-1, :])

MT = np.amax(S, axis=0)
ST = S[-1, :]
payoffs = np.maximum(ST - K, 0)
payoffs[MT >= B_adj] = 0
mc_barrier_u = np.exp(-r * T) * np.mean(payoffs)
mc_barrier_std_err_u = np.exp(-r * T) * ss.sem(payoffs)

print(f"The price of the Up-and-Out call option by Monte Carlo simulation is: {mc_barrier_u:.3f} \
    \nwith standard error: {mc_barrier_std_err_u:.3f}")

The price of the Up-and-Out call option by Monte Carlo simulation is: 0.475     
with standard error: 0.006


## 2. Lookback Options

An option whose payoff is based on the maximum that the underlying asset price attains over some interval of time prior to expiration is called a *lookback option*. We will consider a *floating strike lookback option*, where the payoff is the difference between the maximum asset price over the time between initiation and expiration and the asset price at expiration. The discussion of this option introduces a new type of differential, a differential that is neither $dt$ nor $d \widetilde{W}(t)$. 

### 2.1 Floating Strike Lookback Option

We begin with a geometric Brownian motion asset price, which may be written as

$$ S(t) = S(0) e^{\sigma \widehat{W}(t)}, $$

where, as before, $\widehat{W}(t) = \alpha t + \widetilde{W}(t)$ and 

$$ a = \frac{1}{\sigma} \left(r - \frac{1}{2} \sigma^2 \right). $$

With 

$$ \widehat{M}(t) = \max_{0 \leq u \leq t} \widehat{W}(t), \quad 0 \leq t \leq T, $$

we may write the maximum of the asset price up to time $t$ as

$$ Y(t) = \max_{0 \leq u \leq t} S(u) = S(0) e^{\sigma \widehat{M}(t)}. $$

The lookback option considered here pays off

$$ V(T) = Y(T) - S(T) $$

at expiration time $T$. This payoff is nonnegative because $Y(T) \geq S(T)$. 

Let $t \in [0, T]$ be given. At time $t$, the risk-neutral price of the lookback option is

$$ V(t) = \widetilde{\mathbb{E}} \left[ e^{-r(T-t)} \left( Y(T) - S(T) \right) \mid \mathcal{F}(t) \right]. $$

Because the pair of processes $(S(t), Y(t))$ has the Markov property, there must exist a function $v(t, x, y)$ such that

$$ V(t) = v(t, S(t), Y(t)). $$

Let's consider an example in Python.

In [5]:
# Closed formula
Sy = 100.0     # maximum asset price up to time t (if t = 0, Sy = S0)
z = S0 / Sy    # change of variable

closed_lookback = S0 * (
    (1 + sigma**2 / (2 * r)) * z * ss.norm.cdf(d1(T, z)) + np.exp(-r * T) * ss.norm.cdf(-d2(T, z)) 
    - (sigma**2 / (2 * r)) * np.exp(-r * T) * z**(1 - (2 * r / sigma**2)) * ss.norm.cdf(-d2(T, 1 / z)) 
    - z
)

print(f"The price of the Lookback option by closed formula is: {closed_lookback:.3f}")

The price of the Lookback option by closed formula is: 14.793


In [6]:
# Monte Carlo method
S = np.zeros((N+1, paths))
S[0, :] = S0

Z = ss.norm.rvs(loc=0, scale=1, size=(N,paths), random_state=42)
for i in range(1, N+1):
    S[i, :] = S[i-1, :] * np.exp((r - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * Z[i-1, :])

YT = np.amax(S, axis=0)
ST = S[-1, :]
payoffs = np.maximum(YT - ST, 0)
mc_lookback = np.exp(-r * T) * np.mean(payoffs)
mc_lookback_std_err = np.exp(-r * T) * ss.sem(payoffs)

print(f"The price of the Lookback option by Monte Carlo simulation is: {mc_lookback:.3f} \
    \nwith standard error: {mc_lookback_std_err:.3f}")

The price of the Lookback option by Monte Carlo simulation is: 13.983     
with standard error: 0.032


## 3. Asian Options

An *Asian option* is one whose payoff includes a time average of the underlying asset price. The average may be over the entire time period between initiation and expiration or may be over some period of time that begins later than the initiation of the option and ends with the option's expiration. The average may be from continuous sampling, 

$$ \frac{1}{T} \int_{0}^{T} S(t) \, dt, $$

or may be from discrete sampling,

$$ \frac{1}{m} \sum_{j=1}^{m} S(t_j), $$

where $0 < t_1 < t_2 \cdots < t_m = T$. The primary reason to base an option payoff on an average asset price is to make it more difficult for anyone to significantly affect the payoff by manipulation of the underlying asset price. The price of Asian options is not known in closed form. Therefore, we must either derive PDEs for Asian option prices or use Monte Carlo simulation.

### 3.1 Fixed Strike Asian Call

Once again, we begin with a geometric Brownian motion $S(t)$ given by

$$ dS(t) = rS(t) \, dt + \sigma S(t) \, d \widetilde{W}(t), $$

where $\widetilde{W}(t), 0 \le t \le T$, is a Brownian motion under the risk-neutral measure $\widetilde{\mathbb{P}}$. Consider a *fixed-strike Asian call* whose payoff at time $T$ is

$$ V(T) = \left(\frac{1}{T} \int_0^T S(t) \, dt - K \right)^+, $$

where the strike price $K$ is a nonnegative constant. The price at times $t$ prior to the expiration time $T$ of this call is given by the risk-neutral pricing formula

$$ V(t) = \widetilde{\mathbb{E}} \left[ e^{-r(T-t)} V(T) \mid \mathcal{F}(t) \right], \ 0 \leq t \leq T. $$

The usual iterated conditioning argument shows that

$$ e^{-rt} V(t) = \widetilde{\mathbb{E}} \left[ e^{-rT} V(T) \mid \mathcal{F}(t) \right], \ 0 \leq t \leq T, $$

is a martingale under $\widetilde{\mathbb{P}}$. This is the quantity we wish to compute.

Let's consider an example in Python.

In [7]:
# Monte Carlo method
S = np.zeros((N+1, paths))
S[0, :] = S0

Z = ss.norm.rvs(loc=0, scale=1, size=(N,paths), random_state=42)
for i in range(1, N+1):
    S[i, :] = S[i-1, :] * np.exp((r - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * Z[i-1, :])
    
AT = np.mean(S, axis=0)
payoffs = np.maximum(AT - K, 0)
mc_asian_fixed = np.exp(-r * T) * np.mean(payoffs)
mc_asian_fixed_std_err = np.exp(-r * T) * ss.sem(payoffs)

print(f"The price of the Fixed Strike Asian Call option by Monte Carlo simulation is: {mc_asian_fixed:.3f} \
    \nwith standard error: {mc_asian_fixed_std_err:.3f}")

The price of the Fixed Strike Asian Call option by Monte Carlo simulation is: 5.513     
with standard error: 0.025
